In [2]:
import pandas as pd
import numpy as np
import re
import ast
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel
nltk.download('punkt')
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package punkt to /home/ozzy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/ozzy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
df = pd.read_parquet('data/movies.pq')
df = df[df.original_language=='en'][:20000]
df.sample(10)

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,release_year,return,director
19976,None,0.0,"['Drama', 'Science Fiction', 'Animation']",152795,en,More than two decades after catapulting to sta...,8.534039,"[Pandora Filmproduktion, Entre Chien et Loup, ...","[Belgium, France, Germany, Israel, Luxembourg,...",2013-05-16,...,122.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,None,The Congress,6.4,165.0,2013,0.00,Ari Folman
18868,None,930000.0,['Comedy'],91070,en,"A comedy centered on two best friends, Kim and...",10.114319,"[Stardust Pictures, Dot Dot Dot Productions]","[Canada, United States of America]",2011-06-18,...,100.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A comedy that's a real mother.,L!fe Happens,5.3,66.0,2011,0.03,Kat Coiro
12671,None,14100000.0,"['Crime', 'Mystery', 'Thriller']",12245,en,"At Oxford University, a professor and a grad s...",5.97385,"[La Fabrique de Films, Telecinco Cinema]","[Spain, France, United Kingdom]",2008-01-18,...,107.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,There is no way of finding a single absolute t...,The Oxford Murders,5.7,191.0,2008,0.00,Álex de la Iglesia
2362,Crocodile Dundee Collection,5000000.0,"['Adventure', 'Comedy']",9671,en,When a New York reporter plucks crocodile hunt...,7.791212,"[Paramount Pictures, Rimfire Films]",[Australia],1986-09-26,...,97.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,There's a little of him in all of us.,Crocodile Dundee,6.3,512.0,1986,65.64,Peter Faiman
12557,None,0.0,['Documentary'],18218,en,Director Abby Epstein's controversial document...,1.057283,[],[United States of America],2008-01-01,...,87.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,None,The Business of Being Born,7.3,13.0,2008,0.00,Abby Epstein
24202,None,0.0,"['Action', 'Drama']",19996,en,The story is set in the latter days of World W...,0.741591,[Almighty Dog Productions],[United States of America],2009-05-26,...,100.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,None,Brother's War,3.7,7.0,2009,0.00,Jerry Buteyn
4040,None,0.0,"['Romance', 'Drama']",48217,en,"In 1850, on the isolated French island of Sain...",1.4414,"[Cinémaginaire Inc., Epithete films, France2 C...","[France, Canada]",2000-01-01,...,112.0,"[{'iso_639_1': 'fr', 'name': 'Français'}]",Released,None,The Widow of Saint-Pierre,6.7,11.0,2000,0.00,Patrice Leconte
12310,None,50000000.0,"['Drama', 'Horror', 'Music']",13885,en,"The infamous story of Benjamin Barker, a.k.a S...",10.038401,"[Warner Bros., DreamWorks Pictures, Tim Burton...","[United Kingdom, United States of America]",2007-12-20,...,116.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Never Forget. Never Forgive.,Sweeney Todd: The Demon Barber of Fleet Street,7.0,1745.0,2007,3.04,Tim Burton
3313,None,10000000.0,"['Drama', 'Romance']",41823,en,Opens with Bleek as a child learning to play t...,2.715137,"[Universal Pictures, 40 Acres & A Mule Filmworks]",[United States of America],1990-08-03,...,129.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,None,Mo' Better Blues,6.3,48.0,1990,1.62,Spike Lee
20055,None,0.0,"['Action', 'Thriller']",65515,en,"An ex-Navy SEAL, Michael Adams, (Matthew Reese...",0.733475,"[Mainstay Productions, Matthew Reese Films]",[United States of America],2008-01-01,...,93.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"When Terror Strikes At Home, Blood Will Be Shed",The Eleventh Hour,4.3,4.0,2008,0.00,John Lyde


In [46]:
df['overview'].sample(5)

10676    With a reputation for seducing members of the ...
17250    A passenger of a hijacked airliner who happens...
12529    An insomniac (Stephen Baldwin) who walks the s...
14249    As Australian cinema broke through to internat...
18662    True story of Mary Griffith, gay rights crusad...
Name: overview, dtype: object

In [36]:
df.genres.info()

<class 'pandas.core.series.Series'>
Int64Index: 15000 entries, 0 to 19297
Series name: genres
Non-Null Count  Dtype 
--------------  ----- 
15000 non-null  object
dtypes: object(1)
memory usage: 234.4+ KB


In [81]:
def content(row):
    cont = row.overview
    if row.belongs_to_collection:
        cont += ' ' + row.belongs_to_collection.replace('Collection', '') 
    # Vamos a reemplzar los caracteres que no sean leras por espacios
    cont=re.sub("[^a-zA-Z]"," ",str(cont))
    # Pasamos todo a minúsculas
    cont=cont.lower()
    # Tokenizamos para separar las palabras del cont
    cont=nltk.word_tokenize(cont)
    # Eliminamos las palabras de menos de 3 letras
    #cont = [palabra for palabra in cont if len(palabra)>=3]
    # Sacamos las Stopwords
    # cont = [palabra for palabra in cont if not palabra in stopwords]


    #cont = cont + ast.literal_eval(row.genres)
    #cont.append(row['director'].lower().replace(' ', ''))

    cont = cont  + ast.literal_eval(row.genres)
    #Keep the first 20 words
    #cont= set(cont)
    
    ## Hasta acá Normalizamos, ahora a stemmizar
    
    # Aplicamos la funcion para buscar la raiz de las palabras
    #cont=[stemmer.stem(palabra) for palabra in cont]
    # Por ultimo volvemos a unir el cont
    cont= " ".join(cont)

    return cont

df['content'] = df.apply(content, axis=1)

In [82]:
df.content

0        led by woody andy s toys live happily in his r...
1        when siblings judy and peter discover an encha...
2        a family wedding reignites the ancient feud be...
3        cheated on mistreated and stepped on the women...
4        just when george banks has recovered from his ...
                               ...                        
26083    ger nimo is a year old man devastated by a tra...
26085    american tattoo artist jake sawyer wanders the...
26086    when emily woodrow and her friends happen on a...
26087    a demented handyman comes to the rescue of a y...
26088    pro wrestling legend steve austin stars as a m...
Name: content, Length: 20000, dtype: object

In [83]:
indices = pd.Series(df.index, index=df['content']).drop_duplicates()
content = df['content'].fillna('')

In [84]:
cosine_similarities = None
tfidf = TfidfVectorizer(min_df=2, max_df=0.95)
tfidf_matrix = tfidf.fit_transform(content)

tfidf_matrix = tfidf_matrix.astype('float32')

cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)
#cosine_similarities = cosine_similarity(tfidf_matrix, tfidf_matrix)


In [98]:
# Return indices for the target dataframe column and drop any duplicates
indices = pd.Series(df.index, index=df['content']).drop_duplicates()

# Get the index for the target value
target_index = indices[df.content[0]]

# Get the cosine similarity scores for the target value
cosine_similarity_scores = list(enumerate(cosine_similarities[target_index]))

# Sort the cosine similarities in order of closest similarity
cosine_similarity_scores = sorted(cosine_similarity_scores, key=lambda x: x[1], reverse=True)

for i in cosine_similarity_scores:
    print (df.loc[i[0]].title, i[1])






Toy Story 1.0
The Perfect Holiday 0.50139034
National Lampoon’s European Vacation 0.49765357
The Visit 0.28479695


KeyError: 18884

In [93]:
def get_recommendations(df, column, value, cosine_similarities, limit=10):
    """Return a dataframe of content recommendations based on TF-IDF cosine similarity.
    
    Args:
        df (object): Pandas dataframe containing the text data. 
        column (string): Name of column used, i.e. 'title'. 
        value (string): Name of title to get recommendations for, i.e. 1982 Ferrari 308 GTSi For Sale by Auction
        cosine_similarities (array): Cosine similarities matrix from linear_kernel
        limit (int, optional): Optional limit on number of recommendations to return. 
        
    Returns: 
        Pandas dataframe. 
    """
    
    # Return indices for the target dataframe column and drop any duplicates
    indices = pd.Series(df.index, index=df[column]).drop_duplicates()



    # Get the index for the target value
    target_index = indices[value]

    print(target_index)

    # Get the cosine similarity scores for the target value
    cosine_similarity_scores = list(enumerate(cosine_similarities[target_index]))

    # Sort the cosine similarities in order of closest similarity
    cosine_similarity_scores = sorted(cosine_similarity_scores, key=lambda x: x[1], reverse=True)

    # Return tuple of the requested closest scores excluding the target item and index
    cosine_similarity_scores = cosine_similarity_scores[1:limit+1]


    # Extract the tuple values
    index = (x[0] for x in cosine_similarity_scores)
    scores = (x[1] for x in cosine_similarity_scores)    

    # Get the indices for the closest items
    recommendation_indices = [i[0] for i in cosine_similarity_scores]


    # Get the actutal recommendations
    recommendations = df[column].iloc[recommendation_indices]

    # Return a dataframe
    df = pd.DataFrame(list(zip(index, recommendations, scores)), 
                      columns=['index','recommendation', 'cosine_similarity_score']) 

    return df

In [ ]:
txt = df.content[0]
recommendations = get_recommendations(df, 
                                      'content', 
                                      txt, 
                                      cosine_similarities)
for i in recommendations:
    print (df.loc[i[0]].title, i[1])

In [5]:
print(df.shape)
df.drop_duplicates(subset=['title'], keep='last')
print(df.shape)
df[df.title=='Cars'].id

(20000, 21)
(20000, 21)


11007    920
Name: id, dtype: int64